# `xsec` tutorial (Tools 2020)

In this brief tutorial we'll make a tour of the basic functionality of `xsec`. A manual describing the code structure and usage details is available at [arxiv:2006.16273](https://arxiv.org/abs/2006.16273).

This tutorial can be followed in two ways:
* The simplest way to get started immediately is to click on the Binder link and follow the notebook from within a browser.
* Alternatively, download the notebook and follow the `xsec` installation instructions on [github.com/jeriek/xsec](https://github.com/jeriek/xsec). Depending on your setup, there's a good chance that it's as simple as running `pip install xsec` in a terminal window.

## Installing `xsec`
It's possible to run a shell command from within a Jupyter notebook, by adding an exclamation mark in front of the command. Let's use this to install `xsec` and its dependencies.

In [ ]:
! pip install xsec

## Downloading required data
The `pip` installation **does NOT automatically include the data** required to run `xsec`. To download data after the `pip` installation, one has to execute the following shell command:
```
xsec-download-gprocs [-g GP_DIR] [-t PROCESS_TYPE]
```
The first optional argument `GP_DIR` specifies the name of the (preferably new) directory where the data files will be downloaded and extracted.
If this argument is not specified, a new directory `gprocs` is created in the current working directory. The second optional argument `PROCESS_TYPE` allows for selecting which data to download:

- `gg` (gluino pair production, 220 MB)
- `sg` (1st/2nd gen. squark--gluino pair production, 148 MB)
- `ss` (1st/2nd gen. squark pair production, 1.6 GB)
- `sb` (1st/2nd gen. squark--anti-squark pair production, 766 MB)
- `tb` (3rd gen. squark--anti-squark pair production, 210 MB)
- `all` (everything, 3 GB)

The default option is `all`, but there is no need for such a large download in this tutorial. Get started by downloading the gluino-gluino data. 

In [ ]:
! xsec-download-gprocs -t gg

## First cross-section evaluation
Let's get started immediately by computing the gluino-gluino production cross-section in the case were all squark masses are 500 GeV and the gluino mass is 1 TeV. First, we have to point `xsec` to the directory where the Gaussian process data were downloaded in the previous step.

If no argument `GP_DIR` was entered earlier, the initialisation call requires no argument. Otherwise, specify the keyword `data_dir="GP_DIR"`.

### Initialisation

In [ ]:
import xsec

# Set GP directory choice, default: data_dir="gprocs"
xsec.init()

In this first version of the code, only 13 TeV data is publicly available. Nevertheless, this must be specified:

In [ ]:
# Set center-of-mass energy (in GeV)
xsec.set_energy(13000)

Now, we have to specify that we want the gluino-gluino process. The `load_processes()` function takes as argument a list of all the requested processes, where each process is given as a tuple of two integers: the final-state particle identification numbers (PIDs). The gluino PID is `1000021`. A full list is available in the [PDG](https://pdg.lbl.gov/2019/reviews/rpp2019-rev-monte-carlo-numbering.pdf).  To show the full list of processes available in `xsec`, execute `xsec.list_all_xsec_processes()`. (Only the gluino-gluino data has been downloaded so far, though.)

In [ ]:
# Load GP models for the specified process(es)
processes = [(1000021, 1000021)]
xsec.load_processes(processes)

In [ ]:
xsec.list_all_xsec_processes()

It's normal if the loading step takes a moment, as it requires decompressing the data and performing some matrix multiplications.
This step only needs to be done once at the start of a script. 

### Setting parameters
For this simple first example with degenerate squark-masses, we can just set the required parameters with some convenience functions:

In [ ]:
# Set parameter values
xsec.set_all_squark_masses(500)
xsec.set_gluino_mass(1000)

You can always check the current values of the parameters:

In [ ]:
xsec.get_parameters()

### Evaluation
Everything is set for the evaluation!

In [ ]:
# Evaluate the cross section with the given input parameters
xsec.eval_xsection()

The first output value is the central-scale cross-section value in femtobarn. The other values are, respectively, the relative lower/upper regression error and uncertainties from scale, PDF and $\alpha_s$ variation. As you see at the bottom of the terminal output, the `eval_xsection()` call returns a `numpy` array with all if this information.

`eval_xsection()` takes an optional keyword argument to control how much information is printed in the terminal window. By default, `verbose=2`. For single-line output, try setting `verbose=1`. (`verbose=0` prints nothing, so the return array must be caught or the prediction information is lost.)  

To finish the evaluation procedure, one can run the `finalise()` command. Here, it will only create a `.bib` file with the bibliographic references relevant to the computed results.

In [ ]:
# Finalise the evaluation procedure
xsec.finalise()

## Setting all parameters


Let's first clear all parameters set in the previous example.

In [ ]:
xsec.clear_parameters()

Often, the convenience functions used earlier to set the parameters don't suffice. Then the parameters can be set one by one, using `xsec.set_parameter("parameter_name",value)`, or all at once with a dictionary:

In [ ]:
# Enter dictionary with parameter values
xsec.set_parameters(
    {
        "m1000021": 1000,
        "m1000001": 500,
        "m1000002": 500,
        "m1000003": 500,
        "m1000004": 500,
        "m1000005": 500,
        "m1000006": 500,
        "m2000001": 500,
        "m2000002": 500,
        "m2000003": 500,
        "m2000004": 500,
        "m2000005": 500,
        "m2000006": 500,
        "sbotmix11": 0,
        "stopmix11": 0,
        "mean": 500,
    }
)

We can again evaluate the cross-section:

In [ ]:
xsec.eval_xsection()

Unsurprisingly, the result is the same as earlier. Let's change the $\tilde{d_L}$ mass and see what happens to the cross-section. Remember that you can always check the current values with `get_parameters()`.

In [ ]:
xsec.set_parameter("m1000001", 1500)
xsec.eval_xsection()

This leads to an error! Indeed, since we change the $\tilde{d}_L$ mass, the average mass of the 1st- and 2nd-generation masses is no longer 500 GeV, leading to an inconsistency. The help message indicates that we should update that parameter to 625 GeV. One way to do this is by running `xsec.set_parameter("mean", 625)`. Another option is to simply run 

In [ ]:
xsec.parameters.calc_mean_squark_mass()

Verify that this worked: 

In [ ]:
xsec.get_parameter("mean")

Let's try the cross-section evaluation again!

In [ ]:
xsec.eval_xsection()

## Task
* Clear all parameters.
* Download the squark-gluino production data, and load the $\tilde{g}\tilde{u}_L$ process in `xsec`. 
* Verify which parameters are required for this process, by looking at Table 1 in the `xsec` paper.
* Set the $\tilde{u}_L$ mass and the mean 1st- and 2nd-generation squark mass (`mean`) to 1 TeV, and write a loop varying the gluino mass between 500 GeV and 3 TeV. Ensure that only a single line is printed for each prediction, by choosing the right `verbosity` level.
* Compare your result to the left-hand plot of Fig. 9 in the `xsec` paper.

This covers most of the basic usage of `xsec`! Further instructions on SLHA input, the command-line interface and cache options are described in the accompanying paper. Feel free to open a GitHub issue if you discover a bug or have any question concerning the code. Thanks for following along and good luck hunting for SUSY!